In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from joblib import Parallel, delayed
import pickle
import re
import multiprocessing



In [16]:
search1_all= pd.read_excel('data/search1_all_alternative.xlsx')
link = pd.read_csv('gvkey_cik_link.csv', encoding='latin1')
link_permno = pd.read_csv('cik_permno.csv', encoding='latin1')
link_permno = link_permno[['gvkey', 'LPERMNO']]

C:\Users\TJR\AppData\Local\Temp\ipykernel_9168\3507053983.py:3: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  link_permno = pd.read_csv('cik_permno.csv', encoding='latin1')


In [17]:
# use sequence as unique identifier
search1_all['unique_id'] = search1_all.index

df = pd.merge(search1_all, link, on='cik', how='left')

# create a new column, identify whether date is between link_start_date and link_end_date
df['date'] = pd.to_datetime(df['date'])
df['link_start_date'] = pd.to_datetime(df['link_start_date'], format='%Y-%m-%d', errors='coerce')
df['link_end_date'] = pd.to_datetime(df['link_end_date'], format='%Y-%m-%d', errors='coerce')
df['link_date'] = (df['date'] >= df['link_start_date']) & (df['date'] <= df['link_end_date'])

# for loop each unique id, 1. if there is more than one row, keep the ones with link_date = True 2. if there is only one row, keep it
def filter_link_date(df):
    if len(df) > 1 and df['link_date'].sum() > 0:
        return df[df['link_date']]
    else:
        return df

df = df.groupby('unique_id').apply(filter_link_date).reset_index(drop=True)
print(df.unique_id.nunique())

df.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)

df = pd.merge(df, link_permno, on='gvkey', how='left')

df.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)

6996


C:\Users\TJR\AppData\Local\Temp\ipykernel_9168\2836575751.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('unique_id').apply(filter_link_date).reset_index(drop=True)


In [19]:
df.to_excel('search1_all.xlsx', index=False)

In [ ]:
for year in range(2000,2021):
    df_year = df[df['year'] == year].drop(columns=['unique_id', 'link_start_date', 'link_end_date', 'link_date'])
    df_year.to_csv(f'search1_{year}.csv', index=False)